In [146]:
import math
import numpy as np
import torch
import torch.utils.data as Data
import matplotlib.pyplot as plt

In [147]:
#创建子类
class subDataset(Data.Dataset):
    #初始化，定义数据内容和标签
    def __init__(self, Data, Label):
        self.Data = Data
        self.Label = Label
    #返回数据集大小
    def __len__(self):
        return len(self.Data)
    #得到数据内容和标签
    def __getitem__(self, index):
        data = torch.Tensor(self.Data[index])
        label = torch.Tensor(self.Label[index])
        return data, label


In [148]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class MyDataset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)
    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

In [149]:
from scipy.io import loadmat
#file = "D:\\310 科研\\000 Simulation\\1110_time1000_S6.mat"
file6 = "D:\\310 科研\\000 Simulation\\1111_S6_time1000.mat"
data6 = loadmat(file6)
file12 = "D:\\310 科研\\000 Simulation\\1111_S12_time1000.mat"
data12 = loadmat(file12)
file24 = "D:\\310 科研\\000 Simulation\\1111_S24_time1000.mat"
data24 = loadmat(file24)
""" experiment
X_train = data['Input']
X_train = torch.from_numpy(X_train).float()
y_train = data['Stability']
y_train = y_train.astype(np.float32) / 255
y_train = torch.tensor(y_train)
len_train = len(y_train)
X_test = data['PreInput']
X_test = torch.from_numpy(X_test).float()
y_test = data['PreSta']
y_test = y_test.astype(np.float32) / 255
y_test = torch.tensor(y_test)
len_test = len(y_test) """

#simulation
X6 = data6['Input_Abundance']
X6 = torch.from_numpy(X6).float()
y6 = data6['Std_total']
y6 = y6.T
y6 = torch.from_numpy(y6).float()

X12 = data12['Input_RelativeAbundance']
X12 = torch.from_numpy(X12).float()
y12 = data12['Stability']
y12 = torch.from_numpy(y12).float()

X24 = data24['Input_RelativeAbundance']
X24 = torch.from_numpy(X24).float()
y24= data24['Stability']
y24 = torch.from_numpy(y24).float()

X_train = X6[0:300,:]
y_train = y6[0:300]
#y_train = X6[0:300,0]

X_test = X6[300:1000,:]
y_test = y6[300:1000]
#y_test = X6[300:1000,0]

len_train = len(y_train)
len_test = len(y_test)

# 生成数据
train_data_tensor = X_train
train_target_tensor = y_train
test_data_tensor = X_test
test_target_tensor = y_test

# 将数据封装成Dataset
train_dataset = MyDataset(train_data_tensor, train_target_tensor)
test_dataset = MyDataset(test_data_tensor, test_target_tensor)

In [150]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 10

""" 需不需要随机采样？ 随机采样和shuffle是不是一样的 """
""" # define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, 
                                           sampler = train_sampler, num_workers = num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size,
                                         num_workers = num_workers) """

# prepare data loaders
train_dataloader = DataLoader(dataset = train_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程
test_dataloader = DataLoader(dataset = test_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程
#for data,target in train_dataloader:
#    print(target.size(1))

In [151]:
import torch.nn as nn
import torch.nn.functional as F

# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(54, 1)
        self.fc2 = nn.Linear(54, 54)
    def forward(self,x):
        #x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc1(x))
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=54, out_features=1, bias=True)
  (fc2): Linear(in_features=54, out_features=54, bias=True)
)


In [152]:
# specify loss function
criterion = nn.BCELoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr = 0.1)

In [153]:
# number of epochs to train the model
n_epochs = 100
train_loss_min = np.Inf  # set initial "min" to infinity

for epoch in range(n_epochs):
    train_loss = 0
    # train the model #
    loss = 0
    for data, target in train_dataloader: 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    # calculate average loss over an epoch
    train_loss = train_loss / len_train
    
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    
    # save model
    if train_loss <= train_loss_min:
        print('Train loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        train_loss_min,
        train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss

Epoch: 1 	Training Loss: 0.424133
Train loss decreased (inf --> 0.424133).  Saving model ...
Epoch: 2 	Training Loss: 0.231207
Train loss decreased (0.424133 --> 0.231207).  Saving model ...
Epoch: 3 	Training Loss: 0.169836
Train loss decreased (0.231207 --> 0.169836).  Saving model ...
Epoch: 4 	Training Loss: 0.142397
Train loss decreased (0.169836 --> 0.142397).  Saving model ...
Epoch: 5 	Training Loss: 0.127564
Train loss decreased (0.142397 --> 0.127564).  Saving model ...
Epoch: 6 	Training Loss: 0.118540
Train loss decreased (0.127564 --> 0.118540).  Saving model ...
Epoch: 7 	Training Loss: 0.112637


Train loss decreased (0.118540 --> 0.112637).  Saving model ...
Epoch: 8 	Training Loss: 0.108533
Train loss decreased (0.112637 --> 0.108533).  Saving model ...
Epoch: 9 	Training Loss: 0.105598
Train loss decreased (0.108533 --> 0.105598).  Saving model ...
Epoch: 10 	Training Loss: 0.103423
Train loss decreased (0.105598 --> 0.103423).  Saving model ...
Epoch: 11 	Training Loss: 0.101753
Train loss decreased (0.103423 --> 0.101753).  Saving model ...
Epoch: 12 	Training Loss: 0.100445
Train loss decreased (0.101753 --> 0.100445).  Saving model ...
Epoch: 13 	Training Loss: 0.099404
Train loss decreased (0.100445 --> 0.099404).  Saving model ...
Epoch: 14 	Training Loss: 0.098561
Train loss decreased (0.099404 --> 0.098561).  Saving model ...
Epoch: 15 	Training Loss: 0.097881
Train loss decreased (0.098561 --> 0.097881).  Saving model ...
Epoch: 16 	Training Loss: 0.097327
Train loss decreased (0.097881 --> 0.097327).  Saving model ...
Epoch: 17 	Training Loss: 0.096840
Train loss 

In [154]:

model.load_state_dict(torch.load('model.pt'))
for name, parameters in model.named_parameters():
    print(name, ':', parameters.size())
print(model.fc1.weight)

fc1.weight : torch.Size([1, 54])
fc1.bias : torch.Size([1])
fc2.weight : torch.Size([54, 54])
fc2.bias : torch.Size([54])
Parameter containing:
tensor([[-0.1086,  0.0007, -0.0713, -0.3525, -0.3356, -0.1838, -0.1133,  0.0976,
         -0.2465, -0.1674, -0.5301,  0.0696, -0.2983, -0.3626, -0.3986,  0.0158,
         -0.1299, -0.3705, -0.3369, -0.1123,  0.0098, -0.3401, -0.4739, -0.0810,
          0.1148, -0.3076, -0.3868, -0.3000, -0.1762,  0.0462,  0.3125, -0.3655,
         -0.4192, -0.4167, -0.1766, -0.2212,  0.0481, -0.1947,  0.1107,  0.1448,
         -0.2697, -0.2183, -0.1916, -0.2666,  0.0340,  0.0985, -0.3640,  0.0767,
          0.1735, -0.0436, -0.3005, -0.4003, -0.2734, -0.0422]],
       requires_grad=True)


In [155]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
correct = 0
OUTPUT = []
S_pre = torch.Tensor(len_test,1)
S_true = torch.Tensor(len_test,1)

for i in range(len_test):
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(X_test[i])
    o = output.tolist()
    OUTPUT.append(o)
    # calculate the loss
    loss = criterion(output, y_test[i])
    # update test loss 
    test_loss += loss.item()
    # use the output to predict y
    if(output<0.01): S_pre[i] = 0
    else: S_pre[i] = 1
    if(y_test[i]<0.01): S_true[i] = 0
    else: S_true[i] = 1   
    correct += S_pre[i].eq(S_true[i])

# calculate and print avg test loss
test_loss = test_loss/len_test
correct_ratio = correct/len_test
print('Test Loss: {:.6f}\n'.format(test_loss))
print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
    100*correct_ratio,correct,len_test))


Test Loss: 0.089802


Test Accuracy: 12% (87/700)
